<a href="https://colab.research.google.com/github/Hwismos/capstone-keras-based-model/blob/main/node2vec/node2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/MyDrive/4학년/캡스톤/[인공지능] 실습/[05.08] Keras-node2vec"

# node2vec을 이용한 그래프 표상 학습
- movielens 데이터셋을 이용해 영화에 대한 임베딩을 생성할 수 있는 node2vec 모델을 구현한다.


## Introduction
- 그래프 구조의 객체로부터 유용한 표상을 학습하는 것은 머신러닝 응용에 있어 유용하다. 
    - 특히, 추천 시스템에 이용될 수 있다.
- Graph representation Learning(그래프 표상 학습)은 그래프의 노드들에 대한 임베딩을 학습하는 것을 목표로 한다.
    - 노드 라벨 예측(__node classification__)과 링크 예측(__recommendation__)에 이용될 수 있다.
- node2vec은 그래프의 노드에 대한 저차원 임베딩을 학습하는 것에 효과적인 방법이다. 
    - 그래프의 이웃관계 성질을 최적화하는 것을 목적으로 한다.
    - 이웃한 노드들의 임베딩을 유사하게 학습시키는 것을 목적으로 한다.
- 아이템들이 그래프 구조로 주어진 데이터에 대하여 node2vec은 다음과 같이 동작한다.
    1. random walk를 이용해 아이템 순서들을 생성한다.
    2. 생성한 아이템 순서들에 대해 양성, 음성 학습 예시들을 생성한다.
    3. word2vec 모델을 훈련시켜 아이템들에 대한 임베딩을 학습한다. 
- Movielens 데이터셋은 영화들을 노드로 취급하고 유저들로부터 유사한 레이팅을 받은 영화 간에 간선을 생성하여 그래프 구조로 표현된다.
- 학습된 영화들의 임베딩은 영화 추천에 이용될 수 있다.

In [ ]:
# networks 라이브러리를 필요로 한다.
!pip install networks

## Setup

In [ ]:
import os
from collections import defaultdict
import math
import networkx as nx
import random
from tqdm import tqdm
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

## Download the MovieLens dataset and prepare the data
- Movielens 100k 데이터셋은 610 명의 유저와 9,742개의 영화 정보를 갖고 있다. 
    - 100k개의 간선 정보를 갖고 있다.
- 다운로드된 폴더 중 movies.dat과 ratings.dat 데이터 파일만을 이용한다.

In [ ]:
urlretrieve(
        "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip", "movielens.zip"
)
ZipFile("movielens.zip", "r").extractall()

In [ ]:
# 데이터 전처리를 위해 다운로드한 데이터를 pandas DataFrame에 적재한다.

# movies를 DataFrame에 로드한다.
movies = pd.read_csv("./ml-latest-small/movies.csv")
# "movieId" 문자열을 생성한다.
movies["movieId"] = movies["movieId"].apply(lambda x: f"movie_{x}")

# ratings를 DataFrame에 로드한다.
ratings = pd.read_csv("./ml-latest-small/ratings.csv")
# 'rating'을 float으로 변환한다.
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))
# "movie_id" 문자열을 생성한다.
ratings["movieId"] = ratings["movieId"].apply(lambda x: f"movie_{x}")

print("Movies data shape: ", movies.shape)
print("Ratings data shape: ", ratings.shape)

In [ ]:
# ratings df(DataFrame)와 movies df의 인스턴스 샘플을 점검한다.

ratings.head()

In [ ]:
movies.head()

In [ ]:
# movies df에 대한 두 가지 유틸리티 함수를 구현한다.

def get_movie_title_by_id(movieId):
    return list(movies[movies.movieId == movieId].title)[0]

def get_movie_id_by_title(title):
    return list(movies[movies.title == title].movieId)[0]

## Construct the Movie Graph
- 두 영화가 같은 사람으로부터 min_rating 이상의 레이팅을 받았다면, 두 영화 노드 간에 간선을 생성한다.
- 간선의 가중치는, 두 영화 간 __pointwise mutual information__에 기반하며 이것은 다음과 같이 계산된다.
    - log(xy) - log(x) - log(y) + log(D)
        - xy는 얼마나 많은 유저들이 동시에 x와 y 영화에 min_rating 이상으로 레이팅을 했는지에 따라 결정된다.
        - x는 얼마나 많은 유저들이 영화 x에 min_rating 이상으로 레이팅 했는지에 따라 결정된다.
        - y는 얼마나 많은 유저들이 영화 y에 min_rating 이상으로 레이팅 했는지에 따라 결정된다.
        - D는 min_rating 이상의 레이팅 값을 갖는 영화들의 개수이다.


### Step 1: create the weighted eges between movies.

In [ ]:
min_rating = 5
pair_frequency = defaultdict(int)  # 디폴트 값이 int인 딕셔너리다.
item_frequency = defaultdict(int)

# min_rating 이상의 레이팅 값을 갖는 인스턴스들을 필터링 한다.
rated_movies = ratings[ratings.rating >= min_rating]
# rated_movies.head()

# 유저별로 인스턴스를 grouping한다.
movies_grouped_by_users = list(rated_movies.groupby("userId"))
for group in tqdm(
    movies_grouped_by_users,
    position=0,
    leave=True,
    desc="Cmpute movie rating frequencies",
):
    # 유저에 의해 레이팅된 영화들의 리스트를 가져온다.
    current_movies = list(group[1]["movieId"])

    # 이 부분이 잘 이해가 되지 않는다.
    for i in range(len(current_movies)):
        item_frequency[current_movies[i]] += 1
        for j in range(i+1, len(current_movies)):
            x = min(current_movies[i], current_movies[j])
            y = max(current_movies[i], current_movies[j])
            pair_frequency[(x, y)] += 1

### Step 2: create the graph with the nodes and the edges


In [ ]:
# 노드 간의 간선의 수를 줄이기 위해, 간선의 가중치가 min_weight 이상인 영화 간의 간선들만을 더한다.

min_weight = 10
D = math.log(sum(item_frequency.values()))

# 무방향 movies 그래프를 생성한다.
movies_graph = nx.Graph()
# 영화 간 가중화된 간선을 더한다.
# 간선을 추가하게 되면 자동적으로 영화 노드들은 추가된다.
for pair in tqdm(
    pair_frequency, position=0, leave=True, desc="Creating the movie graph"
):
    x, y = pair
    xy_frequency = pair_frequency[pair]
    x_frequency = item_frequency[x]
    y_frequency = item_frequency[y]
    pmi = math.log(xy_frequency) - math.log(x_frequency) - math.log(y_frequency) + D
    weight = pmi * xy_frequency
    # min_weight 이상의 가중치를 갖는 간선들만을 포함한다.
    if weight >= min_weight:
        movies_graph.add_edge(x, y, weight=weight)

In [ ]:
# 그래프의 노드와 간선의 수를 출력한다.
# 조건에 의해 min_weight 이상의 가중치만을 갖는 간선의 양 끝 노드들만이 추가되었기 때문에 노드들의 개수는 기존의 영화 개수보다 적다.

print("Total number of graph nodes: ", movies_graph.number_of_nodes())
print("Total number of graph edges: ", movies_graph.number_of_edges())

In [ ]:
# 그래프의 노드들의 평균 차수(degree)를 출력한다.
degrees = []
for node in movies_graph.nodes:
    degrees.append(movies_graph.degree[node])

print("Average node degree: ", round(sum(degrees) / len(degrees), 2))

### Step 3: create vocabulary and a mapping from tokens to integer indices

In [ ]:
# vocab은 그래프의 노드(movie IDs)들이다.

vocabulary = ["NA"] + list(movies_graph.nodes)
vocabulary_lookup = {token: idx for idx, token in enumerate(vocabulary)}

## Implement the biased random walk
- random walk는 한 노드로부터 랜덤하게 이웃한 노드로 이동한다.
- 만약 한 간선이 편중되었다면, 편중된 정도에 따라 확률적으로 해당 간선으로 이어진 노드에 도달하게 된다. 
- 이 과정이 num_steps만큼 반복되면 관련된 노드들의 sequence(순서)가 생성되게 된다.
- biased random walk는 BFS와 DFS 사이에서 두 개의 매개변수를 이용해 균형을 맞춘다.
    1. Return parameter(p): 이 값을 이용해 이전 노드에 대한 재방문 가능성을 조절한다. p값을 높게 설정하면 더 다양한 노드들을 방문하는데 반해, 낮게 설정하면 근처 노드들만을 방문한다. 
    2. In-out parameter(q): 이 값을 이용해 멀리 이동할 가능성을 조절한다. q 값이 클수록 지역적으로만 노드가 방문하고, 작을수록 멀리 있는 노드에도 방문한다.

In [ ]:
def next_step(graph, previous, current, p, q):
    neighbors = list(graph.neighbors(current))

    weights = []
    # p와 q를 이용해 이웃 노드에 대한 간선의 가중치를 조정한다.
    for neighbor in neighbors:
        if neighbor == previous:
            # 이전 노드로 돌아갈 가능성을 조정한다.
            weights.append(graph[current][neighbor]["weight"] / p)
        elif graph.has_edge(neighbor, previous):
            # 근처 노드를 방문할 가능성은 간선의 가중치와 같다.
            weights.append(graph[current][neighbor]["weight"])
        else:
            # 새로운 노드로 이동할 가능성을 조정한다.
            weights.append(graph[current][neighbor]["weight"] / q)
    
    # 각 이웃 노드를 방문할 확률을 계산한다.
    weight_sum = sum(weights)
    probabilities = [weight / weight_sum for weight in weights]
    # 확률적으로 다음으로 방문할 이웃 노드를 고른다.
    next = np.random.choice(neighbors, size=1, p=probabilities)[0]
    return next

def random_walk(graph, num_walks, num_steps, p, q):
    walks = []
    nodes = list(graph.nodes())
    # random walk를 여러 번 반복적으로 실행시킨다.
    for walk_iteration in range(num_walks):
        random.shuffle(nodes)

        for node in tqdm(
            nodes,
            position=0,
            leave=True,
            desc=f"Random walks iteration {walk_iteration + 1} of {num_walks}",
        ):
            # 그래프의 랜덤한 한 노드로부터 이동을 시작한다.
            walk = [node]
            # num_steps만큼 랜덤하게 이동한다.
            while len(walk) < num_steps:
                current = walk[-1]
                previous = walk[-2] if len(walk) > 1 else None
                # 다음으로 방문할 노드를 계산한다.
                next = next_step(graph, previous, current, p, q)
                walk.append(next)
            walk = [vocabulary_lookup[token] for token in walk]
            walks.append(walk)
    return walks

## Generating traing data using the biased random walk

In [ ]:
# p와 q를 조정할 수 있다.

# return parameter
p = 1
# in-out parameter
q = 1
# random walks의 반복 횟수
num_walks = 5
# random walks의 스텝 수
num_steps = 10
walks = random_walk(movies_graph, num_walks, num_steps, p, q)

print("\n\nNumber of walks generated: ", len(walks))

## Generate positive and negative examples
- skip-gram 모델을 훈련시키기 위해 생성된 walks를 이용해 positive, negative 훈련 샘플을 생성한다. 
- 각 샘플은 다음과 같은 특징을 갖는다.
    1. target: walk 순서에 있는 한 영화이다.
    2. context: walk 순서에 있는 다른 영화이다.
    3. weight: target과 context가 walk sequences에서 나타나는 횟수이다.
    4. label: 두 영화가 walk sequences 샘플에 있다면 1, 아니면 0으로 라벨링한다.

### Generate examples

In [ ]:
def generate_examples(sequences, window_size, num_negative_samples, vocabulary_size):
    example_weights = defaultdict(int)
    # Iterate over all sequences (walks).
    for sequence in tqdm(
        sequences,
        position=0,
        leave=True,
        desc=f"Generating postive and negative examples",
    ):
        # Generate positive and negative skip-gram pairs for a sequence (walk).
        pairs, labels = keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocabulary_size,
            window_size=window_size,
            negative_samples=num_negative_samples,
        )
        for idx in range(len(pairs)):
            pair = pairs[idx]
            label = labels[idx]
            target, context = min(pair[0], pair[1]), max(pair[0], pair[1])
            if target == context:
                continue
            entry = (target, context, label)
            example_weights[entry] += 1

    targets, contexts, labels, weights = [], [], [], []
    for entry in example_weights:
        weight = example_weights[entry]
        target, context, label = entry
        targets.append(target)
        contexts.append(context)
        labels.append(label)
        weights.append(weight)

    return np.array(targets), np.array(contexts), np.array(labels), np.array(weights)


num_negative_samples = 4
targets, contexts, labels, weights = generate_examples(
    sequences=walks,
    window_size=num_steps,
    num_negative_samples=num_negative_samples,
    vocabulary_size=len(vocabulary),
)

In [ ]:
# 결과물의 shape을 출력한다.

print(f"Targets shape: {targets.shape}")
print(f"Contexts shape: {contexts.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Weights shape: {weights.shape}")

### Convert the data into tf.data.Dataset objects

In [ ]:
batch_size = 1024


def create_dataset(targets, contexts, labels, weights, batch_size):
    inputs = {
        "target": targets,
        "context": contexts,
    }
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels, weights))
    dataset = dataset.shuffle(buffer_size=batch_size * 2)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


dataset = create_dataset(
    targets=targets,
    contexts=contexts,
    labels=labels,
    weights=weights,
    batch_size=batch_size,
)

## Train the skip-gram model
- skip-gram 모델은 심플한 이진 분류 모델이며 다음과 같이 동작한다.
    1. target 영화에 대한 임베딩이 검색된다.
    2. context 영화에 대한 임베딩이 검색된다.
    3. 두 임베딩 간의 내적 연산이 실행된다.
    4. label과 시그모이드 활성화 함수를 거친 두 임베딩의 내적 값이 비교된다.
    5. binary corssentropy loss가 사용된다.

In [ ]:
learning_rate = 0.001
embedding_dim = 50
num_epochs = 10

### Implement the model

In [ ]:
def create_model(vocabulary_size, embedding_dim):

    inputs = {
        "target": layers.Input(name="target", shape=(), dtype="int32"),
        "context": layers.Input(name="context", shape=(), dtype="int32"),
    }
    # 아이템 임베딩을 초기화한다.
    # 양의 정수(인덱스)들을 고정된 사이즈의 dense vector로 치환한다.
    embed_item = layers.Embedding(
        input_dim=vocabulary_size,
        output_dim=embedding_dim,
        embeddings_initializer="he_normal",
        embeddings_regularizer=keras.regularizers.l2(1e-6),
        name="item_embeddings",
    )
    # 타겟에 대한 임베딩을 검색한다.
    target_embeddings = embed_item(inputs["target"])
    # context에 대한 임베딩을 검색한다. 
    context_embeddings = embed_item(inputs["context"])
    # target과 context 간의 내적 값을 계산한다.
    logits = layers.Dot(axes=1, normalize=False, name="dot_similarity")(
        [target_embeddings, context_embeddings]
    )
    # 모델 인스턴스를 생성한다.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

### Train the model

In [ ]:
# 모델 인스턴스를 만든 뒤 컴파일 한다.

model = create_model(len(vocabulary), embedding_dim)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
)

In [ ]:
# model을 도식화하여 출력한다.
keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
)

In [ ]:
# 데이터셋에 대해 모델을 훈련시킨다.

history = model.fit(dataset, epochs=num_epochs)

In [ ]:
# 학습에 대한 히스토리를 그래프로 표현한다.

plt.plot(history.history["loss"])
plt.ylabel("losss")
plt.xlabel("epoch")
plt.show()

## Analyze the learnt embeddings

In [ ]:
movie_embeddings = model.get_layer("item_embeddings").get_weights()[0]

# 저장된 SavedModel 파일을 이용해 모델을 로드하고 기존과 동일한 결과를 반환하는지를 확인한다.
# movie_embeddings = new_model.get_layer("item_embeddings").get_weights()[0]  

print("Embeddings shape:", movie_embeddings.shape)

### Find related movies

In [ ]:
# 몇가지 영화들로 구성된 query_movies 리스트를 정의한다.

query_movies = [
    "Matrix, The (1999)",
    "Star Wars: Episode IV - A New Hope (1977)",
    "Lion King, The (1994)",
    "Terminator 2: Judgment Day (1991)",
    "Godfather, The (1972)",
]

In [ ]:
# query_movies 리스트의 영화들에 대한 임베딩을 가져 온다.

query_embeddings = []

for movie_title in query_movies:
    movieId = get_movie_id_by_title(movie_title)
    token_id = vocabulary_lookup[movieId]
    movie_embedding = movie_embeddings[token_id]
    query_embeddings.append(movie_embedding)

query_embeddings = np.array(query_embeddings)
# print(query_embeddings)

In [ ]:
# query_movies의 영화들의 임베딩과 다른 모든 영화 간의 임베딩 간의 코사인 similarity를 계산한다. 
# 이 중, 유사도가 가장 높은 k를 구한다.

# Module: tf.linalg
# 선형대수학적 연산을 수행해주는 라이브러리다.
similarities = tf.linalg.matmul(
    tf.math.l2_normalize(query_embeddings),
    tf.math.l2_normalize(movie_embeddings),
    transpose_b=True,
)

_, indices = tf.math.top_k(similarities, k=5)
indices = indices.numpy().tolist()

In [ ]:
# query_movies 안에 있는 영화들과 가장 높은 연관성을 갖는 영화들을 출력한다.

for idx, title in enumerate(query_movies):
    print(title)
    print("".rjust(len(title), "-"))
    similar_tokens = indices[idx]
    for token in similar_tokens:
        similar_movieId = vocabulary[token]
        similar_title = get_movie_title_by_id(similar_movieId)
        print(f"- {similar_title}")
    print()

### Visualize the embeddings using the Embedding Projector

In [ ]:
import io

# tsv 파일은 tab으로 컬럼을 분리한다.comma로 컬럼을 분리하는 csv와 유사하다.
out_v = io.open("embeddings.tsv", "w", encoding="utf-8")
out_m = io.open("metadata.tsv", "w", encoding="utf-8")

for idx, movie_id in enumerate(vocabulary[1:]):
    movie_title = list(movies[movies.movieId == movie_id].title)[0]
    vector = movie_embeddings[idx]
    out_v.write("\t".join([str(x) for x in vector]) + "\n")
    out_m.write(movie_title + "\n")

out_v.close()
out_m.close()

## Save and Load

In [ ]:
tf.keras.saving.save_model(
    model, './my_model', overwrite=True, save_format=None
)

In [ ]:
new_model = tf.keras.models.load_model('./my_model')

# new_model의 구조가 기존 모델과 동일함을 확인한다.
keras.utils.plot_model(
    new_model,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
)